### for designing HD mutants MPRA library

In [12]:
import os, sys, warnings, re, glob2, itertools, string

import numpy as np
import pandas as pd
from functools import reduce
import scipy
from scipy import stats

### I housekeeping

In [25]:
# set up working directories (do not run twice)
base_dir = os.path.split(os.getcwd())[0]
sys.path.insert(0, os.path.join(os.getcwd(),"utils"))
os.chdir(base_dir)
scriptdir=os.path.join(base_dir, "scripts")

In [27]:
from utils import specseq_plot_utils, specseq_ewm_utils, specseq_predict_occp, sequence_annotator, mpra_CRE_utils
specseq_plot_utils.set_manuscript_params() # max 7pt

### II. definition of different peaksets (use strict differential analysis stats instead of clustering only)

In [9]:
# CRX ChIP annotation matrix
chip_full_matrix = pd.read_csv(os.path.join(base_dir, "peaksets", "hdmuts_chip_compiled_matrix.tsv"), sep="\t", header=0)

# lfc 0.85 ~ FC 1.8
lfc_th=0.85
fdr_th=0.01
k88n_gained = chip_full_matrix.loc[lambda df: (df["chip.k88n.lfc"]>lfc_th)&(df["chip.k88n.fdr"]<fdr_th), :].reset_index(drop=True).copy()
k88n_lost = chip_full_matrix.loc[lambda df: (df["chip.k88n.lfc"]<-lfc_th)&(df["chip.k88n.fdr"]<fdr_th), :].reset_index(drop=True).copy()
e80a_gained = chip_full_matrix.loc[lambda df: (df["chip.e80a.lfc"]>lfc_th)&(df["chip.e80a.fdr"]<fdr_th), :].reset_index(drop=True).copy()
e80a_lost = chip_full_matrix.loc[lambda df: (df["chip.e80a.lfc"]<-lfc_th)&(df["chip.e80a.fdr"]<fdr_th), :].reset_index(drop=True).copy()
print(f"{len(k88n_gained.index)}, {len(k88n_lost.index)}, {len(e80a_gained.index)}, {len(e80a_lost.index)}")


466, 5422, 214, 309


### take away the selected retinal gene CREs from the peaksets

In [10]:
retinalGene_CRE = pd.read_csv(os.path.join(base_dir, "peaksets", "retinalGenes.csv"), sep=",")
retinalGene_array = retinalGene_CRE["peak.id"].unique() # save all peak id in an array

In [11]:
k88n_gained = k88n_gained.loc[lambda df: ~df["peak.id"].isin(retinalGene_array), :].copy()
k88n_lost = k88n_lost.loc[lambda df: ~df["peak.id"].isin(retinalGene_array), :].copy()
e80a_gained = e80a_gained.loc[lambda df: ~df["peak.id"].isin(retinalGene_array), :].copy()
e80a_lost = e80a_lost.loc[lambda df: ~df["peak.id"].isin(retinalGene_array), :].copy()
print(f"{len(k88n_gained.index)}, {len(k88n_lost.index)}, {len(e80a_gained.index)}, {len(e80a_lost.index)}")

465, 5396, 213, 309


In [12]:
k88n_gained.to_csv(os.path.join(base_dir, "peaksets", "k88n_gained_regions.bed"), sep="\t", index=False, header=True)
k88n_lost.to_csv(os.path.join(base_dir, "peaksets", "k88n_lost_regions.bed"), sep="\t", index=False, header=True)
e80a_gained.to_csv(os.path.join(base_dir, "peaksets", "e80a_gained_regions.bed"), sep="\t", index=False, header=True)
e80a_lost.to_csv(os.path.join(base_dir, "peaksets", "e80a_lost_regions.bed"), sep="\t", index=False, header=True)

### III. generate full CRE list, center on summit, and take 134bp (+66/-67 from summit (0))

In [13]:
mpraAllCRE = chip_full_matrix.loc[:,["peak.id","seqnames","start","end"]].copy()

In [14]:
newCREs = retinalGene_CRE.loc[lambda df: df["peak.id"]=="n.a.",["Seqnames","Start","End"]].reset_index(drop=True)
newCREs["peak.id"] = ["peak."]+pd.Series(str(9833+x) for x in range(8))
newCREs = newCREs.iloc[:,[3,0,1,2]]
newCREs = newCREs.rename(columns={"Seqnames": "seqnames", "Start": "start", "End": "end"})

In [15]:
mpraAllCRE = pd.concat([mpraAllCRE,newCREs], ignore_index=True).copy()
mpraAllCRE["summit"] = round((mpraAllCRE["start"]+mpraAllCRE["end"])/2)
mpraAllCRE["start"] = mpraAllCRE["summit"]-67
mpraAllCRE["end"] = mpraAllCRE["summit"]+66
mpraAllCRE["width"] = mpraAllCRE["end"] - mpraAllCRE["start"] +1
mpraAllCRE["strand"] = "+"

In [16]:
mpraAllCRE.head()

,peak.id,seqnames,start,end,summit,width,strand
0,peak.1,chr1,4357711.0,4357844.0,4357778.0,134.0,+
1,peak.2,chr1,4358542.0,4358675.0,4358609.0,134.0,+
2,peak.3,chr1,4360266.0,4360399.0,4360333.0,134.0,+
3,peak.4,chr1,4383772.0,4383905.0,4383839.0,134.0,+
4,peak.5,chr1,4802559.0,4802692.0,4802626.0,134.0,+


In [14]:
mpraAllCRE.to_csv(os.path.join(base_dir, "sequences", "mpraAllCRE.tsv"), sep="\t", index=False, header=True)

### IV. extract the fasta sequences using BSgenome mm10 package in R (run in specseq env)

In [18]:
inputBed=os.path.join(base_dir, "sequences", "mpraAllCRE.tsv")
outputFasta=os.path.join(base_dir,"sequences","mpraAllCRE.fa")

In [20]:
!Rscript ./coordinate_to_fasta.R "{inputBed}" "peak.id" "{outputFasta}"

System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to create bus connection: Host is down
Warning message:
In system("timedatectl", intern = TRUE) :
  running command 'timedatectl' had status 1
[1] "Reading coordinates from /mnt/v/yqzheng/qiaoer/PhD Thesis/Experiment/MPRA/hdmuts_library/sequences/mpraAllCRE.tsv"
[1] "Using column peak.id as fasta names"
[1] "Writing fasta sequences to /mnt/v/yqzheng/qiaoer/PhD Thesis/Experiment/MPRA/hdmuts_library/sequences/test.mpraAllCRE.fa"
ha! this is the end of the coordinate_to_fasta.R script!any warnings:system("timedatectl", intern = TRUE)


### V. scan dimeric HD motifs with FIMO

In [32]:
meme_dir = os.path.join(base_dir, "meme", "all_chip_pwm.meme")
fimo_th = 2.5E-3 # as in Ryan's eLife paper
#fimo_th = 1.0E-3 # as in Drew's Genome Research paper
fimo_meta = os.path.join(scriptdir, "mpra_fimo_meta.csv")

In [33]:
fimo_sample_list = pd.read_csv(fimo_meta, sep=",", header=0)
fimo_sample_list

,sampleName,inputFA,markovBG,outputDir
0,allCRE,allCRE/mpraAllCRE.fa,allCRE/mpraAllCRE_background,allCRE_fimo
1,allCRE_fimo,allCRE_fimo/allCRE_fimo.k88n_olap.MEME.2.maske...,allCRE_fimo/allCRE_fimo_background,allCRE_fimo2
2,allCRE_fimo2,allCRE_fimo2/allCRE_fimo2.k88n_olap.MEME.2.mas...,allCRE_fimo2/allCRE_fimo2_background,allCRE_fimo3
3,allCRE_fimo3,allCRE_fimo3/allCRE_fimo3.k88n_olap.MEME.2.mas...,allCRE_fimo3/allCRE_fimo2_background,allCRE_fimo4
4,dimerMutCRE,dimerMutCRE/allCRE.k88n_olap.MEME.2.mutated.fa,dimerMutCRE/dimerMutCRE_fimo_background,dimerMutCRE_fimo
5,monoMutCRE,monoMutCRE/allCRE.Crx.MA0467.1.mutated.fa,monoMutCRE/monoMutCRE_fimo_background,monoMutCRE_fimo
6,monoMutCRE_fimo,monoMutCRE_fimo/allCRE.Crx.MA0467.1.mutated.fa,monoMutCRE_fimo/monoMutCRE_fimo_background,monoMutCRE_fimo2
7,monoMutCRE_fimo2,monoMutCRE_fimo2/allCRE.Crx.MA0467.1.mutated.fa,monoMutCRE_fimo2/monoMutCRE_fimo2_background,monoMutCRE_fimo3
8,monoMutCRE,monoMutCRE/allCRE.CRX_Corbo.mutated.fa,monoMutCRE/monoMutCRE_fimo_background,monoMutCRE_fimo
9,monoMutCRE_fimo,monoMutCRE_fimo/allCRE.CRX_Corbo.mutated.fa,monoMutCRE_fimo/monoMutCRE_fimo_background,monoMutCRE_fimo2


In [19]:
# copy the newly generated fasta to fimo folder
new_dir = os.path.join(base_dir, f"fimo_{fimo_th}", os.path.split(fimo_sample_list.iloc[0,1])[0].split("/")[-1])
!cp "{outputFasta}" "{new_dir}"

In [20]:
!bash ./meme_fimo_scanning.sh "{scriptdir}" "{base_dir}" "{meme_dir}" "{fimo_th}" "{fimo_meta}" 1

working directory: /mnt/v/yqzheng/qiaoer/VSCode_yiqiao/SPEC-SEQ/scripts
query fasta: /mnt/v/yqzheng/qiaoer/PhD Thesis/Experiment/MPRA/hdmuts_library/fimo_0.0025/allCRE/mpraAllCRE.fa
9829 134 134 134.0 1317086
Scanning with threshold 0.0025
FIMO output will be written to /mnt/v/yqzheng/qiaoer/PhD Thesis/Experiment/MPRA/hdmuts_library/fimo_0.0025/allCRE_fimo
ha! this is the end of the script!


### go through FIMO to mark all dimeric HD motifs to N and rescan to make sure all motifs are marked

In [21]:
# read all sequences in fasta file as pd.Series with fasta header/identifier as index
raw_fasta = {}
f = os.path.join(base_dir, f"fimo_{fimo_th}", fimo_sample_list.iloc[0,1])
print("reading fasta records from " + os.path.split(f)[1])
raw_fasta[fimo_sample_list.iloc[0,0]] = sequence_annotator.read_fasta(f)
raw_fasta = pd.Series(raw_fasta)
print([x for x in raw_fasta.index])

reading fasta records from mpraAllCRE.fa
['allCRE']


In [22]:
raw_fasta[0]

label
peak.1       TTTTAAGATAATAAAGGTAGCCATAGCAGACAAGTGCGTGAGTAGC...
peak.2       ATCCACAAAGGACAAGCTGAAGATTGCCATGCTCTGGAAGACTTGA...
peak.3       GGATATGCAACCTGCTTGTTTCACGTAAACAAATGTCTTTGGATTT...
peak.4       GTTCCTGTGTGTTTGTTTCCCTGCACACACAGGCTCAGCAGCACAT...
peak.5       AAACTCTGTCTGAAAAACCATAAAAGAAAAAGAAAGATGTAGCCTC...
                                   ...                        
peak.9836    TGAGACTCTGAACTATCCTAAGCCTCCCAAAGACAAAGTCCCAGAT...
peak.9837    CGGCGGGAGCTGCCAGCTTTTTGGAATTCCTAATCGCTCCTGGCCC...
peak.9838    ATGAAGTAGATATTACCAAATTGCTTTTTCAGCATCCATTTAGATA...
peak.9839    TCACCCTAATCCCTCTTTCAAAATGTACTATCCAATTCCATTCTGG...
peak.9840    CCGCCCACCGCTTCGCGACCGGAAGCCGACCGTTTCCCGGGCGACC...
Length: 9829, dtype: object

In [23]:
readin_fa=raw_fasta[0].index.tolist()
set(mpraAllCRE["peak.id"]) - set(readin_fa)

set()

In [24]:
# retrieve and parse raw fimo results
raw_fimo_score = {}
parsed_fimo_score = {}
f = os.path.join(base_dir, f"fimo_{fimo_th}", fimo_sample_list.iloc[1,0], "fimo.tsv")
name = os.path.split(f)[0].split("/")[-1]
print('reading and parsing ' + f)
# read the raw fimo output
raw_fimo_score[name] = pd.read_csv(f, sep="\t", header=0)[:-3] # drop the last three row
# parse the result table
fimo_df = sequence_annotator.parse_raw_fimo(f)
# name the index column
fimo_df.index.name = "label"
# update sumary score dictionary
parsed_fimo_score[name] = fimo_df

# conver dictionary to series for easy access
raw_fimo_score = pd.Series(raw_fimo_score)
parsed_fimo_score = pd.Series(parsed_fimo_score)
print([x for x in parsed_fimo_score.index])

reading and parsing /mnt/v/yqzheng/qiaoer/PhD Thesis/Experiment/MPRA/hdmuts_library/fimo_0.0025/allCRE_fimo/fimo.tsv
['allCRE_fimo']


In [25]:
# save parsed fimo results to file
specseq_predict_occp.save_df(parsed_fimo_score[0], os.path.join(base_dir, f"fimo_{fimo_th}", fimo_sample_list.iloc[1,0], "parsed_fimo.tsv"))

In [26]:
# 1-indexed nucleotide position dictionaries
dimer_core = {1:"T", 2:"A", 3:"A", 9:"T", 10:"T", 11:"A"} # k88n_olap.MEME.2
corbo_mono_core = {2:"T", 3:"A", 4:"A"} # CRX_Corbo
crx_mono_core = {8:"T", 9:"T", 10:"A"} # JASPAR
dimer_mutant_core = {3:"C", 9:"G"}
corbo_mutant_core = {4:"C"}
crx_mutant_core = {8:"G"}

In [27]:
motif = "k88n_olap.MEME.2"
name = raw_fasta.index[0]

masked_fasta = {}
masked_fimo_score = {}

print(f"masking {motif} in {name} fasta sequences")
masked_fimo_score[f"{name}.{motif}"], masked_fasta[f"{name}.{motif}"] = mpra_CRE_utils.find_and_mask_motif(raw_fasta[0], raw_fimo_score[0], motif_name=motif, coremotif_dict=dimer_core)

masked_fasta = pd.Series(masked_fasta)
masked_fimo_score = pd.Series(masked_fimo_score)

print([x for x in masked_fasta.index])

masking k88n_olap.MEME.2 in allCRE fasta sequences
['allCRE.k88n_olap.MEME.2']


In [28]:
len(masked_fimo_score["allCRE.k88n_olap.MEME.2"].index)

2578

In [29]:
# write masked fasta to file
sequence_annotator.write_fasta(masked_fasta["allCRE.k88n_olap.MEME.2"], os.path.join(base_dir, f"fimo_{fimo_th}", fimo_sample_list.iloc[1,1]))

### run FIMO again to mark any remaining dimeric motifs

In [30]:
!bash ./meme_fimo_scanning.sh "{scriptdir}" "{base_dir}" "{meme_dir}" "{fimo_th}" "{fimo_meta}" 2

working directory: /mnt/v/yqzheng/qiaoer/VSCode_yiqiao/SPEC-SEQ/scripts
query fasta: /mnt/v/yqzheng/qiaoer/PhD Thesis/Experiment/MPRA/hdmuts_library/fimo_0.0025/allCRE_fimo/allCRE_fimo.k88n_olap.MEME.2.masked.fa
9829 134 134 134.0 1317086
Scanning with threshold 0.0025
FIMO output will be written to /mnt/v/yqzheng/qiaoer/PhD Thesis/Experiment/MPRA/hdmuts_library/fimo_0.0025/allCRE_fimo2
ha! this is the end of the script!


In [31]:
# read all sequences in fasta file as pd.Series with fasta header/identifier as index
f = os.path.join(base_dir, f"fimo_{fimo_th}", fimo_sample_list.iloc[1,1])
print("reading fasta records from " + os.path.split(f)[1])
raw_fasta[fimo_sample_list.iloc[1,0]] = sequence_annotator.read_fasta(f)
print([x for x in raw_fasta.index])

reading fasta records from allCRE_fimo.k88n_olap.MEME.2.masked.fa
['allCRE', 'allCRE_fimo']


In [32]:
# retrieve raw fimo results
f = os.path.join(base_dir, f"fimo_{fimo_th}", fimo_sample_list.iloc[2,0], "fimo.tsv")
name = os.path.split(f)[0].split("/")[-1]
print('reading and parsing ' + f)
# read the raw fimo output
raw_fimo_score[name] = pd.read_csv(f, sep="\t", header=0)[:-3] # drop the last three row
# parse the result table
fimo_df = sequence_annotator.parse_raw_fimo(f)
# name the index column
fimo_df.index.name = "label"
# update sumary score dictionary
parsed_fimo_score[name] = fimo_df
print([x for x in raw_fimo_score.index])

reading and parsing /mnt/v/yqzheng/qiaoer/PhD Thesis/Experiment/MPRA/hdmuts_library/fimo_0.0025/allCRE_fimo2/fimo.tsv
['allCRE_fimo', 'allCRE_fimo2']


In [33]:
# save parsed fimo results to file
specseq_predict_occp.save_df(parsed_fimo_score[1], os.path.join(base_dir, f"fimo_{fimo_th}", fimo_sample_list.iloc[2,0], "parsed_fimo.tsv"))

In [34]:
motif = "k88n_olap.MEME.2"
name = raw_fasta.index[1]

print(f"masking {motif} in {name} fasta sequences")
masked_fimo_score[f"{name}.{motif}"], masked_fasta[f"{name}.{motif}"] = mpra_CRE_utils.find_and_mask_motif(raw_fasta[1], raw_fimo_score[1], motif_name=motif, coremotif_dict=dimer_core)

print([x for x in masked_fasta.index])

masking k88n_olap.MEME.2 in allCRE_fimo fasta sequences
['allCRE.k88n_olap.MEME.2', 'allCRE_fimo.k88n_olap.MEME.2']


In [35]:
len(masked_fimo_score["allCRE_fimo.k88n_olap.MEME.2"].index)

11

In [36]:
# write masked fasta to file
sequence_annotator.write_fasta(masked_fasta["allCRE_fimo.k88n_olap.MEME.2"], os.path.join(base_dir, f"fimo_{fimo_th}", fimo_sample_list.iloc[2,1]))

### run FIMO again to mark any remaining dimeric motifs

In [37]:
!bash ./meme_fimo_scanning.sh "{scriptdir}" "{base_dir}" "{meme_dir}" "{fimo_th}" "{fimo_meta}" 3

working directory: /mnt/v/yqzheng/qiaoer/VSCode_yiqiao/SPEC-SEQ/scripts
query fasta: /mnt/v/yqzheng/qiaoer/PhD Thesis/Experiment/MPRA/hdmuts_library/fimo_0.0025/allCRE_fimo2/allCRE_fimo2.k88n_olap.MEME.2.masked.fa
9829 134 134 134.0 1317086
Scanning with threshold 0.0025
FIMO output will be written to /mnt/v/yqzheng/qiaoer/PhD Thesis/Experiment/MPRA/hdmuts_library/fimo_0.0025/allCRE_fimo3
ha! this is the end of the script!


In [38]:
# read all sequences in fasta file as pd.Series with fasta header/identifier as index
f = os.path.join(mpraout_dir, f"fimo_{fimo_th}", fimo_sample_list.iloc[2,1])
print("reading fasta records from " + os.path.split(f)[1])
raw_fasta[fimo_sample_list.iloc[2,0]] = sequence_annotator.read_fasta(f)
print([x for x in raw_fasta.index])

reading fasta records from allCRE_fimo2.k88n_olap.MEME.2.masked.fa
['allCRE', 'allCRE_fimo', 'allCRE_fimo2']


In [39]:
# retrieve raw fimo results
f = os.path.join(base_dir, f"fimo_{fimo_th}", fimo_sample_list.iloc[3,0], "fimo.tsv")
name = os.path.split(f)[0].split("/")[-1]
print('reading and parsing ' + f)
# read the raw fimo output
raw_fimo_score[name] = pd.read_csv(f, sep="\t", header=0)[:-3] # drop the last three row
# parse the result table
fimo_df = sequence_annotator.parse_raw_fimo(f)
# name the index column
fimo_df.index.name = "label"
# update sumary score dictionary
parsed_fimo_score[name] = fimo_df
print([x for x in raw_fimo_score.index])

reading and parsing /mnt/v/yqzheng/qiaoer/PhD Thesis/Experiment/MPRA/hdmuts_library/fimo_0.0025/allCRE_fimo3/fimo.tsv
['allCRE_fimo', 'allCRE_fimo2', 'allCRE_fimo3']


In [40]:
# save parsed fimo results to file
specseq_predict_occp.save_df(parsed_fimo_score[2], os.path.join(base_dir, f"fimo_{fimo_th}", fimo_sample_list.iloc[3,0], "parsed_fimo.tsv"))

In [41]:
motif = "k88n_olap.MEME.2"
name = raw_fasta.index[2]

print(f"masking {motif} in {name} fasta sequences")
masked_fimo_score[f"{name}.{motif}"], masked_fasta[f"{name}.{motif}"] = mpra_CRE_utils.find_and_mask_motif(raw_fasta[2], raw_fimo_score[2], motif_name=motif, coremotif_dict=dimer_core)

print([x for x in masked_fasta.index])

masking k88n_olap.MEME.2 in allCRE_fimo2 fasta sequences
['allCRE.k88n_olap.MEME.2', 'allCRE_fimo.k88n_olap.MEME.2', 'allCRE_fimo2.k88n_olap.MEME.2']


In [42]:
masked_fimo_score["allCRE_fimo2.k88n_olap.MEME.2"]

,motif_id,motif_alt_id,sequence_name,start,stop,strand,score,p-value,q-value,matched_sequence,match


In [43]:
# write masked fasta to file #_fimo and _fimo2 fasta should be the same since no more matches were found
sequence_annotator.write_fasta(masked_fasta["allCRE_fimo2.k88n_olap.MEME.2"], os.path.join(base_dir, f"fimo_{fimo_th}", fimo_sample_list.iloc[3,1]))

#### Now there's no more dimeric HD motifs passing FIMO threshold and match core motifs. Theoretically, we can consider the rest of the monomeric motif matching to be clean

### VII. look at motif content

In [44]:
CRX_motif_count_stringent = {}

# count monomeric CRX sites after masking all dimeric sites
motif = "Crx.MA0467.1"
print(" ".join(["counting", motif, "in", raw_fasta.index[2]]))
CRX_motif_count_stringent[motif] = mpra_CRE_utils.count_motif_occur(raw_fasta["allCRE_fimo2"], raw_fimo_score["allCRE_fimo3"], motif_name=motif, coremotif_dict=crx_mono_core)

counting Crx.MA0467.1 in allCRE_fimo2


In [45]:
motif = "CRX_Corbo"
print(" ".join(["counting", motif, "in", raw_fasta.index[2]]))
CRX_motif_count_stringent[motif] = mpra_CRE_utils.count_motif_occur(raw_fasta["allCRE_fimo2"], raw_fimo_score["allCRE_fimo3"], motif_name=motif, coremotif_dict=corbo_mono_core)

counting CRX_Corbo in allCRE_fimo2


In [46]:
motif_column = "k88n_olap.DREME.1"
print(" ".join(["counting", motif_column, "in", raw_fasta.index[2]]))
CRX_motif_count_stringent["k88n_mono"] = mpra_CRE_utils.count_motif_occur(raw_fasta["allCRE_fimo2"], raw_fimo_score["allCRE_fimo3"], motif_name=motif_column, coremotif_dict={5:"T",6:"T",7:"A"})

counting k88n_olap.DREME.1 in allCRE_fimo2


In [47]:
# count dimeric motifs
motif_column = "k88n_olap.MEME.2"
print(" ".join(["counting", motif_column, "in", raw_fasta.index[0]]))
CRX_motif_count_stringent["k88n_olap1"] = mpra_CRE_utils.count_motif_occur(raw_fasta["allCRE"], raw_fimo_score["allCRE_fimo"], motif_name=motif_column, coremotif_dict=dimer_core)

counting k88n_olap.MEME.2 in allCRE


In [48]:
# count dimeric motifs
print(" ".join(["counting", motif_column, "in", raw_fasta.index[1]]))
CRX_motif_count_stringent["k88n_olap2"] = mpra_CRE_utils.count_motif_occur(raw_fasta["allCRE_fimo"], raw_fimo_score["allCRE_fimo2"], motif_name=motif_column, coremotif_dict=dimer_core)

counting k88n_olap.MEME.2 in allCRE_fimo


In [49]:
# count NRL motif
motif_column="NRL.MA0842.2"
print(" ".join(["counting", motif_column, "in", raw_fasta.index[2]]))
CRX_motif_count_stringent["NRL"] = mpra_CRE_utils.count_motif_occur(raw_fasta["allCRE_fimo2"], raw_fimo_score["allCRE_fimo3"], motif_name=motif_column, coremotif_dict=None)

counting NRL.MA0842.2 in allCRE_fimo2


In [50]:
# count AP-1 sites
motif_column="FOS::JUN.MA0099.2"
print(" ".join(["counting", motif_column, "in", raw_fasta.index[2]]))
CRX_motif_count_stringent["AP1"] = mpra_CRE_utils.count_motif_occur(raw_fasta["allCRE_fimo2"], raw_fimo_score["allCRE_fimo3"], motif_name=motif_column, coremotif_dict=None)


counting FOS::JUN.MA0099.2 in allCRE_fimo2


In [51]:
CRX_motif_count_stringent = pd.Series(CRX_motif_count_stringent)

In [52]:
for i,name in zip(CRX_motif_count_stringent.index.tolist(), ["HDmono", "CRX_Corbo", "N50", "HDdimer1", "HDdimer2", "NRL", "AP1"]):
    CRX_motif_count_stringent[i] = CRX_motif_count_stringent[i].rename(columns={"motif_count":name})

In [53]:
# merge all dataframes into one
data_merge = reduce(lambda left, right:
                     pd.merge(left , right, left_index=True, right_index=True),
                     CRX_motif_count_stringent.tolist())
data_merge["HDdimer"] = data_merge["HDdimer1"]+data_merge["HDdimer2"]
data_merge = data_merge[["CRX_Corbo", "N50", "HDmono", "HDdimer", "NRL", "AP1"]].astype('int32')
data_merge.index.name="peak.id"

In [54]:
# write to file
data_merge.to_csv(os.path.join(base_dir, "motifs", f"allCRE_motifcount.fimo{fimo_th}.tsv"), sep="\t", header=True, index=True)

In [56]:
def get_motif_stats(motifcount_df):

    total = len(motifcount_df)
    count_dict = {}
    count_dict["Total CREs"]=total
    # at least one CRX site
    sites=sum((motifcount_df["CRX_Corbo"]>0)|(motifcount_df["N50"]>0)|(motifcount_df["HDdimer"]>0))
    count_dict["At least one CRX site"]=sites
    # Dimeric only
    sites=sum((motifcount_df["CRX_Corbo"]==0)&(motifcount_df["N50"]==0)&(motifcount_df["HDdimer"]>0))
    count_dict["Dimeric HD site only"]=sites
    # Monomeric only
    sites=sum(((motifcount_df["CRX_Corbo"]>0)|(motifcount_df["N50"]>0))&(motifcount_df["HDdimer"]==0))
    count_dict["Monomeric HD site only"]=sites
    # K mono only
    sites=sum((motifcount_df["CRX_Corbo"]>0)&(motifcount_df["HDdimer"]==0))
    count_dict["WT CRX site only"]=sites
    # Q mono only
    sites=sum((motifcount_df["N50"]>0)&(motifcount_df["HDdimer"]==0))
    count_dict["K88N CRX site only"]=sites
    # K+D
    sites=sum((motifcount_df["CRX_Corbo"]>0)&(motifcount_df["HDdimer"]>0))
    count_dict["Co-occurred WT CRX and dimeric HD sites"]=sites
    # Q+D
    sites=sum((motifcount_df["N50"]>0)&(motifcount_df["HDdimer"]>0))
    count_dict["Co-occurred K88N CRX and dimeric HD sites"]=sites
    # K+Q
    sites=sum((motifcount_df["CRX_Corbo"]>0)&(motifcount_df["N50"]>0))
    count_dict["Co-occurred WT and K88N CRX sites"]=sites
    # K+Q+D
    sites=sum((motifcount_df["CRX_Corbo"]>0)&(motifcount_df["N50"]>0)&(motifcount_df["HDdimer"]>0))
    count_dict["Co-occurred WT, K88N, and dimeric sites"]=sites
    # K+NRL
    sites=sum((motifcount_df["CRX_Corbo"]>0)&(motifcount_df["NRL"]>0))
    count_dict["Co-occurred WT CRX and NRL sites"]=sites
    # Q+NRL
    sites=sum((motifcount_df["N50"]>0)&(motifcount_df["NRL"]>0))
    count_dict["Co-occurred k88N CRX and NRL sites"]=sites
    # D+NRL
    sites=sum((motifcount_df["HDdimer"]>0)&(motifcount_df["NRL"]>0))
    count_dict["Co-occurred dimeric HD and NRL sites"]=sites
    # NRL OR AP-1
    sites=sum((motifcount_df["AP1"]>0)|(motifcount_df["NRL"]>0))
    count_dict["NRL or AP1 sites"]=sites
    # NRL
    sites=sum((motifcount_df["NRL"]>0)&(motifcount_df["AP1"]==0))
    count_dict["NRL site only"]=sites
    # AP1
    sites=sum((motifcount_df["AP1"]>0)&(motifcount_df["NRL"]==0))
    count_dict["AP1 site only"]=sites

    count_df = pd.DataFrame.from_dict(count_dict, orient='index', columns=['count'])
    count_df.index.name="Category"
    count_df["Perc"] = count_df["count"]/total
    count_df["Perc"] = count_df["Perc"].round(decimals=4)*100

    return count_df

In [57]:
# background motif content
bg_motif_content = get_motif_stats(data_merge)
#display(bg_motif_content)

### check motif content for different categories of peaksets

In [58]:
diffbound_peaks=list(set(k88n_gained["peak.id"].tolist()+e80a_gained["peak.id"].tolist()+e80a_lost["peak.id"].tolist()+retinalGene_CRE["peak.id"].tolist()))
diffbound_CREs = data_merge.loc[lambda df: df.index.isin(diffbound_peaks),:]

In [61]:
# motif content of differentially bound peaks
db_motif_content = get_motif_stats(diffbound_CREs)
#display(db_motif_content)

In [62]:
k88nlost_CREs = data_merge.loc[lambda df: (df.index.isin(k88n_lost["peak.id"].tolist()))&(~df.index.isin(diffbound_peaks)),:]

In [65]:
# motif content of differentially bound peaks
klost_motif_content = get_motif_stats(k88nlost_CREs)
#display(klost_motif_content)

In [66]:
notdiffbound_CREs = data_merge.loc[lambda df: ~((df.index.isin(diffbound_peaks))|(df.index.isin(k88n_lost["peak.id"].tolist()))),:]

In [69]:
# motif content of differentially bound peaks
notdb_motif_content = get_motif_stats(notdiffbound_CREs)
#display(notdb_motif_content)

In [70]:
# compile all motif content dataframes into a multi-index dataframe
d = {"All": bg_motif_content, "DB": db_motif_content, "KLost": klost_motif_content, "NotDB": notdb_motif_content }
complete_motif_content = pd.concat(d.values(), axis=1, keys=d.keys())
display(complete_motif_content)

All            DB         KLost  \
                                          count    Perc count    Perc count   
Category                                                                      
Total CREs                                 9829  100.00  1031  100.00  5163   
At least one CRX site                      8545   86.94   968   93.89  4413   
Dimeric HD site only                        258    2.62    71    6.89   132   
Monomeric HD site only                     7125   72.49   556   53.93  3846   
WT CRX site only                           6820   69.39   493   47.82  3737   
K88N CRX site only                         2572   26.17   311   30.16  1243   
Co-occurred WT CRX and dimeric HD sites    1017   10.35   253   24.54   413   
Co-occurred K88N CRX and dimeric HD sites   589    5.99   261   25.32   140   
Co-occurred WT and K88N CRX sites          2711   27.58   421   40.83  1252   
Co-occurred WT, K88N, and dimeric sites     444    4.52   173   16.78   118   
Co-occurred WT CRX and NRL sites           4084   41.55   405   39.28  2142   
Co-occurred k88N CRX and NRL sites         1782   18.13   318   30.84   779   
Co-occurred dimeric HD and NRL sites        680    6.92   203   19.69   264   
NRL or AP1 sites                           7021   71.43   793   76.92  3584   
NRL site only                              2493   25.36   277   26.87  1313   
AP1 site only                              2062   20.98   225   21.82  1033   

                                                  NotDB          
                                             Perc count    Perc  
Category                                                         
Total CREs                                 100.00  3635  100.00  
At least one CRX site                       85.47  3164   87.04  
Dimeric HD site only                         2.56    55    1.51  
Monomeric HD site only                      74.49  2723   74.91  
WT CRX site only                            72.38  2590   71.25  
K88N CRX site only                          24.08  1018   28.01  
Co-occurred WT CRX and dimeric HD sites      8.00   351    9.66  
Co-occurred K88N CRX and dimeric HD sites    2.71   188    5.17  
Co-occurred WT and K88N CRX sites           24.25  1038   28.56  
Co-occurred WT, K88N, and dimeric sites      2.29   153    4.21  
Co-occurred WT CRX and NRL sites            41.49  1537   42.28  
Co-occurred k88N CRX and NRL sites          15.09   685   18.84  
Co-occurred dimeric HD and NRL sites         5.11   213    5.86  
NRL or AP1 sites                            69.42  2644   72.74  
NRL site only                               25.43   903   24.84  
AP1 site only                               20.01   804   22.12

In [71]:
complete_motif_content.to_csv(os.path.join(base_dir, "motifs", f"compiled_motifcontent.fimo{fimo_th}.tsv"), sep="\t", index=True)

### compile binding, fimo score, and annotation into one dataframe

In [72]:
mpraAnnot_df = pd.merge(mpraAllCRE, data_merge, left_on="peak.id", right_on="peak.id", how="outer")

In [73]:
mpraAnnot_df["annotation"]=np.nan
mpraAnnot_df[["annotation"]] = mpraAnnot_df[["annotation"]] .mask(mpraAnnot_df["peak.id"].isin(retinalGene_CRE["peak.id"].tolist()), 'RetinalGene', inplace=False) 
mpraAnnot_df[["annotation"]] = mpraAnnot_df[["annotation"]] .mask(mpraAnnot_df["peak.id"].isin(e80a_gained["peak.id"].tolist()), 'EGain', inplace=False)
mpraAnnot_df[["annotation"]] = mpraAnnot_df[["annotation"]] .mask(mpraAnnot_df["peak.id"].isin(e80a_lost["peak.id"].tolist()), 'ELost', inplace=False)
mpraAnnot_df[["annotation"]] = mpraAnnot_df[["annotation"]] .mask(mpraAnnot_df["peak.id"].isin(k88n_gained["peak.id"].tolist()), 'KGain', inplace=False)
mpraAnnot_df[["annotation"]] = mpraAnnot_df[["annotation"]] .mask(mpraAnnot_df["peak.id"].isin(k88nlost_CREs.index.tolist()), 'KLost', inplace=False)
mpraAnnot_df[["annotation"]] = mpraAnnot_df[["annotation"]] .mask(mpraAnnot_df["peak.id"].isin(notdiffbound_CREs.index.tolist()), 'NotDB', inplace=False)

In [74]:
mpraAnnot_df = mpraAnnot_df.merge(raw_fasta[0].to_frame(name="FASTA"), left_on="peak.id", right_on="label", how="outer")

In [75]:
mpraAnnot_df.head()

,peak.id,seqnames,start,end,summit,width,strand,CRX_Corbo,N50,HDmono,HDdimer,NRL,AP1,annotation,FASTA
0,peak.1,chr1,4357711.0,4357844.0,4357778.0,134.0,+,1,2,0,0,2,1,KLost,TTTTAAGATAATAAAGGTAGCCATAGCAGACAAGTGCGTGAGTAGC...
1,peak.2,chr1,4358542.0,4358675.0,4358609.0,134.0,+,1,0,1,0,0,1,KLost,ATCCACAAAGGACAAGCTGAAGATTGCCATGCTCTGGAAGACTTGA...
2,peak.3,chr1,4360266.0,4360399.0,4360333.0,134.0,+,3,2,3,0,2,1,RetinalGene,GGATATGCAACCTGCTTGTTTCACGTAAACAAATGTCTTTGGATTT...
3,peak.4,chr1,4383772.0,4383905.0,4383839.0,134.0,+,3,1,0,0,3,1,NotDB,GTTCCTGTGTGTTTGTTTCCCTGCACACACAGGCTCAGCAGCACAT...
4,peak.5,chr1,4802559.0,4802692.0,4802626.0,134.0,+,0,0,0,1,0,1,ELost,AAACTCTGTCTGAAAAACCATAAAAGAAAAAGAAAGATGTAGCCTC...


In [76]:
mpraAnnot_df.to_csv(os.path.join(base_dir, "peaksets", f"allCRE_annotation.fimo{fimo_th}.tsv"), sep="\t", index=False)

In [77]:
mpraAnnot_df.loc[lambda df: df["HDdimer"]>=3,:]

,peak.id,seqnames,start,end,summit,width,strand,CRX_Corbo,N50,HDmono,HDdimer,NRL,AP1,annotation,FASTA
16,peak.17,chr1,10024896.0,10025029.0,10024963.0,134.0,+,0,1,0,4,0,1,KLost,CTGAGTTTTGGTAACTGGGTTAAAAGTGTTACTTGAGCTTCTTTTC...
652,peak.653,chr10,19096590.0,19096723.0,19096657.0,134.0,+,1,1,1,3,0,0,NotDB,GGCCCATATGATGTCATACTTTGATTATCCACTTGATCCCACAGAG...
1211,peak.1212,chr11,35873995.0,35874128.0,35874062.0,134.0,+,0,0,0,3,2,2,KLost,CTCCGCTAAACCACTTAAGGAGATTAATTTGTCTGGTGACTGAACG...
2154,peak.2158,chr12,85841229.0,85841362.0,85841296.0,134.0,+,2,0,1,3,0,1,KLost,TGTATCTTAATCTCTGCAGCCTATGGATATAAACCTTTTATGGTAA...
3505,peak.3511,chr15,94383359.0,94383492.0,94383426.0,134.0,+,3,0,0,4,0,0,KGain,TTCTATCAAGTCAAGACTAAAACTGATTCTTGCCCTTTATTCTTAG...
3673,peak.3679,chr16,31652711.0,31652844.0,31652778.0,134.0,+,1,1,0,3,0,1,KGain,CAACTTCTGCCTTTCTGTTTCGGGGGCTTCATTTAACTGGTTTATG...
4207,peak.4214,chr17,71480789.0,71480922.0,71480856.0,134.0,+,1,3,1,3,0,2,NotDB,TGCTAATGACTTTATCACTCCCTATGGCCAGTCTCTAAACGCCTTA...
4310,peak.4317,chr18,10576206.0,10576339.0,10576273.0,134.0,+,0,1,0,3,1,1,KGain,TAGAGATAGATTGCTCACAATTCCTTCTATTTTAATTTTAAGCCTA...
4631,peak.4638,chr18,84372236.0,84372369.0,84372303.0,134.0,+,1,3,0,3,1,0,NotDB,CATCTGTCTTTAATACTCTTATTACAAGGCAGTAAGCAAGGAACTA...
4827,peak.4834,chr19,33335274.0,33335407.0,33335341.0,134.0,+,2,1,0,3,0,1,NotDB,TTTTGATTAAGCAAGGCTCTCAGTTATAATTCTCATGGATTTAAGT...


### scan and calculate occupancy of all peaks

In [ ]:
# retrieve MEME pwm
hdmuts_meme_pwms = specseq_ewm_utils.read_meme_files(os.path.join(base_dir,"all_chip_pwm.meme"))
# retrieve MEME pwm
prtf_meme_pwms = specseq_ewm_utils.read_meme_files(os.path.join(base_dir,"meme","photoreceptorAndEnrichedMotifs.meme"))
# retrieve Lee 2010 CRX EMSA binding models
lee_pwm = specseq_ewm_utils.read_meme_files(os.path.join(base_dir,"meme","Lee_2010_pwm.meme"))
lee_ewm = lee_pwm.apply(lambda x: specseq_ewm_utils.pwm_to_ewm(x, pseudocount=0.0001, temp=25, normalize=False))

#### convert pwm to ewm

In [107]:
hdmuts_meme_ewms = hdmuts_meme_pwms.apply(lambda x: specseq_ewm_utils.pwm_to_ewm(x, pseudocount=0.0001, temp=25, normalize=False))
prtf_meme_ewms = prtf_meme_pwms.apply(lambda x: specseq_ewm_utils.pwm_to_ewm(x, pseudocount=0.0001, temp=25, normalize=False))

#### load full fasta records

In [6]:
allCRE_fasta = sequence_annotator.read_fasta(os.path.join(base_dir, "sequences", "mpraAllCRE.fa"))
maskedallCRE_fasta = sequence_annotator.read_fasta(os.path.join(base_dir, "fimo_0.0025", "allCRE_fimo3", "allCRE_fimo3.k88n_olap.MEME.2.masked.fa"))

#### define library design

In [7]:
# Define library design (update as new libraries added)
lib_designs = {
    # for automatically adding filter sequences
    "M":"TAANNN",
    "Mrev":"NNNTTA",
    "MGGG":"TAANNNGGG",
    "P3TAAT":"TAATNNNATTA",
    "P5TAAT":"TAATNNGNNATTA",
}
consensus_dicts = {
    "wt":{"M":"TAATCC", "Mrev":"GGATTA", "P3TAAT":"TAATCCGATTA"},
    "e80a":{"M":"TAATCC", "Mrev":"GGATTA", "P3TAAT":"TAATCGGATTA"},
    "r90w":{"M":"TAATCC", "Mrev":"GGATTA", "P3TAAT":"TAATCCGATTA"},
    "k88n":{"M":"TAATTA", "Mrev":"TAATTA", "P3TAAT":"TAATGTCATTA"},
}

In [108]:
try_ewms = pd.concat([hdmuts_meme_ewms,lee_ewm])

In [109]:
try_ewms.index

Index(['wt_olap.DREME.1', 'e80a_olap.DREME.1', 'k88n_olap.MEME.2',
       'k88n_olap.DREME.1', 'k88n_gain.MEME.1', 'e80a_loss.MEME.3',
       'CRX_Corbo', 'Crx.MA0467.1', 'NRL.MA0842.2', 'FOS::JUN.MA0099.2',
       'Lee_2010_CRX'],
      dtype='object')

In [110]:
# predict occupancy with Lee 2010 CRX EWM and ChIP inferred EWM
mu = 0.1

# prepare design and consensus series based on ewms in the current run
design_list = [None]*len(try_ewms)
consensus_list = [None]*len(try_ewms)

# calculate occupancy landscape
occupancy_df = specseq_predict_occp.all_seq_total_occupancy(allCRE_fasta, try_ewms, design_list, consensus_list, mu=mu, convert_ewm=False)
# Save to file
specseq_predict_occp.save_df(occupancy_df, os.path.join(base_dir, "predicted_occp", "allCRE.unmasked.occupancy.mu" + str(mu) + ".tsv"))

# calculate occupancy landscape
occupancy_df = specseq_predict_occp.all_seq_total_occupancy(maskedallCRE_fasta, try_ewms, design_list, consensus_list, mu=mu, convert_ewm=False)
# Save to file
specseq_predict_occp.save_df(occupancy_df, os.path.join(base_dir, "predicted_occp", "allCRE.masked.occupancy.mu" + str(mu) + ".tsv"))

using mu equals 0.1 for calculation
using mu equals 0.1 for calculation


In [111]:
# predict occupancy with Lee 2010 CRX EWM and ChIP inferred EWM
mu = 9

# prepare design and consensus series based on ewms in the current run
design_list = [None]*len(try_ewms)
consensus_list = [None]*len(try_ewms)

# calculate occupancy landscape
occupancy_df = specseq_predict_occp.all_seq_total_occupancy(allCRE_fasta, try_ewms, design_list, consensus_list, mu=mu, convert_ewm=False)
# Save to file
specseq_predict_occp.save_df(occupancy_df, os.path.join(base_dir, "predicted_occp", "allCRE.unmasked.occupancy.mu" + str(mu) + ".tsv"))

# calculate occupancy landscape
occupancy_df = specseq_predict_occp.all_seq_total_occupancy(maskedallCRE_fasta, try_ewms, design_list, consensus_list, mu=mu, convert_ewm=False)
# Save to file
specseq_predict_occp.save_df(occupancy_df, os.path.join(base_dir, "predicted_occp", "allCRE.masked.occupancy.mu" + str(mu) + ".tsv"))

using mu equals 9 for calculation
using mu equals 9 for calculation


### VIII. processing the 20 controls from published study (Friedman et al., 2021, eLife)

In [ ]:
control_CREs = pd.read_csv(os.path.join(base_dir, "peaksets", "positiveControls.tsv"), sep="\t", header=0)
control_fasta = sequence_annotator.read_fasta(os.path.join(base_dir, "sequences", "positiveControls.fasta"))

#### Ryan's library is 164bp so I need to trim them to 134bp, remove the first 15bp and the last 15bp

In [ ]:
control_CREs["new_start"] = control_CREs["start"] + 16
control_CREs["new_stop"] = control_CREs["stop"] - 15

In [ ]:
trimmed_control_CREs = control_CREs.loc[:,["sequence_name", "new_start", "new_stop"]].copy()

In [ ]:
trimmed_control_CREs.to_csv(os.path.join(base_dir, "peaksets", "positiveControls.trimmed.tsv"), sep="\t", header=False, index=False)

In [ ]:
control_fasta = control_fasta.to_frame().rename(columns={0:"fulllength"})
control_fasta["trimmed"] = control_fasta["fulllength"].apply(lambda fasta: fasta[15:-15])
trimmed_control_fasta = control_fasta["trimmed"]

In [ ]:
sequence_annotator.write_fasta(trimmed_control_fasta, os.path.join(base_dir, "sequences", "positiveControls.trimmed.fa"))